# Identificación de ciudades principales

#### ¿Qué hacemos en este cuaderno?
Como los datos del [NOTI-SINADEF] están desagregados hasta el nivel distrital, necesitamos saber qué distritos forman parte de cada una de las *ciudades principales* para luego agregarlos y poder analizar los fallecimientos por Covid-19 a nivel de ciudades.

Para ello, en este cuaderno limpiaremos la cartografía de INEI que contiene los límites de cada *ciudad principal*. 

¿Por qué? Porque los límites marcados por INEI a veces toman partes muy pequeñas de distritos en los que existen otras ciudades (no necesariamente *principales*)

![Ciudad de Nueva Cajamarca en el departamento de San Martín](imagenes/NuevaCajamarca_DPTOsanmartin.jpeg)
![imagen](imagenes/limitesdeciudades.jpeg)

#### ¿Qué procedimiento seguimos?
Para hacerlo, (1) 'partimos' las manchas urbanas por distritos (a cada partición les llamaremos 'pedacitos'), (2) calculamos cuánta población vive en cada 'pedacito' de mancha urbana, y (3) calculamos cuánta población vive en toda la mancha urbana. Con estos tres datos podremos tener dos columnas que nos sirvarán para hacer el filtro de limpiado: el porcentaje poblacional que representa cada 'pedacito' de la mancha urbana a la que pertenece (*PR*), y el porcentaje poblacional que representa cada 'pedacito' del distrito al que pertenece (*PR_2*).

Cuando consigo estas dos variables, de manera manual reviso en QGIS algunos 'pedacitos' para ver qué *PR* y *PR_2* tienen. Luego de revisar algunas manchas urbanas veo que los 'pedacitos' que deberían irse (por ser muy pequeños) tienen un 'PR' cercano a 8%.

Como la revisión fue visual, es posible que si los elimino manualmente se me escape alguno. Por tanto, establezco un umbral para todas las manchas urbanas. Ese umbral lo establezco cercano al valor observado: el percentil 45, que corresponde al valor de 8.28145614%.

Sin embargo, veo que con ese filtro aplicado aún hay errores en algunas manchas urbanas. Por tanto, añado el *PR_2* de 30%, también después de una revisión visual de aquellos 'pedacitos' que creo deben mantenerse en el geodataframe [^1].

#### ¿Qué obtenemos?
Al final de este proceso podremos saber (1) qué distritos forman parte de cada *ciudad principal* y (2) qué población tiene cada *ciudad principal*.

#### Datos utilizados
Para lograrlo utilizaremos tres geodataframes obtenidos del procesamiento en PyQIS:
1. Un gdf que tiene partidas las manchas urbanas por distritos y la población que la conforma (*4-ciudades_pob.geojson*). No nos interesa su geometría, sino sus datos.
2. Un gdf que tiene solo las manchas urbanas repartidas por distritos (*3-ciudades_interseccion.geojson*). Aquí si nos interesa su geometría.
3. Un df con la población de cada distrito del país, obtenida directamente de INEI.


[^1]: Esta también es un umbral arbitrario. Puede someterse a discusión.

[NOTI-SINADEF]:https://www.datosabiertos.gob.pe/dataset/fallecidos-por-covid-19-ministerio-de-salud-minsa
[INEI produjo para el Censo de 2017]: https://www.geogpsperu.com/2020/07/manzanas-y-poblacion-de-todo-el-peru.html

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np

#1.Calcularemos cuánta población tiene cada 'pedacito' de las manchas urbanas
ciudades=gpd.read_file("data/4-ciudades_pob.geojson")
ciudades_pedacitos=ciudades.loc[:,['ID','T_TOTAL']]
ciudades_pedacitos=ciudades_pedacitos.groupby("ID").sum()
ciudades_pedacitos.head()

,T_TOTAL
ID,
1.0,8346.0
2.0,61967.0
3.0,11302.0
4.0,11579.0
5.0,34355.0


In [3]:
#2.Ahora cuánta población tiene toda la mancha urbana que conforman las ciudades.
ciudades_enteras=ciudades.loc[:,['CIUDAD','T_TOTAL']]
ciudades_enteras=ciudades_enteras.groupby('CIUDAD').sum()
ciudades_enteras.head()

,T_TOTAL
CIUDAD,
ABANCAY,70313.0
ANDAHUAYLAS,57236.0
AREQUIPA,990673.0
AYACUCHO,220210.0
AYAVIRI,21370.0


In [4]:
#3.Importamos el gdf con las manchas urbanas de las Ciudades Principales repartidas 
#por distrito
ciudades_i=gpd.read_file("data/3-ciudades_interseccion.geojson")
ciudades_i.head()

,CODDPTO,NOMBDPTO,CIUDAD,layer,path,NOMBDIST,CODUBIGEO,IDPROV,NOMBPROV,ID,geometry
0,03,APURIMAC,ABANCAY,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,TAMBURCO,30109,0301,ABANCAY,1,"MULTIPOLYGON (((730284.004 8494704.893, 730288..."
1,03,APURIMAC,ABANCAY,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,ABANCAY,30101,0301,ABANCAY,2,"MULTIPOLYGON (((730726.099 8493090.736, 730727..."
2,03,APURIMAC,ANDAHUAYLAS,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,TALAVERA,30216,0302,ANDAHUAYLAS,3,"MULTIPOLYGON (((667519.696 8491559.078, 667530..."
3,03,APURIMAC,ANDAHUAYLAS,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,SAN JERONIMO,30213,0302,ANDAHUAYLAS,4,"MULTIPOLYGON (((676109.501 8490005.382, 676102..."
4,03,APURIMAC,ANDAHUAYLAS,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,ANDAHUAYLAS,30201,0302,ANDAHUAYLAS,5,"MULTIPOLYGON (((671729.911 8490283.357, 671731..."


In [31]:
#4.Le añadimos los cálculos hechos previamente
#4.1. Primero, la población de toda la Mancha Urbana (PobMU)
ciudades_nuevo = gpd.GeoDataFrame(ciudades_i.merge(ciudades_enteras, on='CIUDAD', how='left'))
ciudades_nuevo.rename(columns={'T_TOTAL':'PobMU'},inplace=True)

#4.2. Luego, la población solo del 'pedacito' de mancha urbana (PobParcial)
ciudades_nuevo = gpd.GeoDataFrame(ciudades_nuevo.merge(ciudades_pedacitos, on='ID', how='left'))
ciudades_nuevo.rename(columns={'T_TOTAL':'PobParcial'},inplace=True)

#4.3.Esto nos permitirá saber qué porcentaje de población hay en cada 'pedacito'
#respecto del total de población de la Mancha Urbana de las Ciudades Principales
ciudades_nuevo['PR']= ciudades_nuevo['PobParcial']/ciudades_nuevo['PobMU']*100

ciudades_nuevo.head()

,CODDPTO,NOMBDPTO,CIUDAD,layer,path,NOMBDIST,CODUBIGEO,IDPROV,NOMBPROV,ID,geometry,PobMU,PobParcial,PR
0,03,APURIMAC,ABANCAY,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,TAMBURCO,30109,0301,ABANCAY,1,"MULTIPOLYGON (((730284.004 8494704.893, 730288...",70313.0,8346.0,11.869782
1,03,APURIMAC,ABANCAY,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,ABANCAY,30101,0301,ABANCAY,2,"MULTIPOLYGON (((730726.099 8493090.736, 730727...",70313.0,61967.0,88.130218
2,03,APURIMAC,ANDAHUAYLAS,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,TALAVERA,30216,0302,ANDAHUAYLAS,3,"MULTIPOLYGON (((667519.696 8491559.078, 667530...",57236.0,11302.0,19.746314
3,03,APURIMAC,ANDAHUAYLAS,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,SAN JERONIMO,30213,0302,ANDAHUAYLAS,4,"MULTIPOLYGON (((676109.501 8490005.382, 676102...",57236.0,11579.0,20.230275
4,03,APURIMAC,ANDAHUAYLAS,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,ANDAHUAYLAS,30201,0302,ANDAHUAYLAS,5,"MULTIPOLYGON (((671729.911 8490283.357, 671731...",57236.0,34355.0,60.023412


In [29]:
#5.Importamos la info de la población por distritos

#dist_p=gpd.read_file('data/0-manzanas_centros.geojson')
#dist_p.rename(columns={'UBIGEO':'CODUBIGEO'},inplace=True)#Necesitamos que coincida el nombre con el df anterior
#dist_p.CODUBIGEO=dist_p.CODUBIGEO.astype(int)             #Y necesitamos que sea del mismo tipo:un número

dist_p = pd.read_csv("rawdata/(PROCESADO) Poblacion Urbana y Total-Distritos INEI 2017.csv")
dist_p.rename(columns={'UBIGEO':'CODUBIGEO'},inplace=True)
dist_p=dist_p.loc[:,['CODUBIGEO','PobTot']]
dist_p=dist_p.groupby("CODUBIGEO").sum()
dist_p.head()

,PobTot
CODUBIGEO,
10101,32589.0
10102,262.0
10103,1136.0
10104,642.0
10105,585.0


In [32]:
#6.Unimos el df obtenido en el paso 4 con el obtenido en el paso 5
ciudades_nuevo=gpd.GeoDataFrame(ciudades_nuevo.merge(dist_p,on='CODUBIGEO'))
ciudades_nuevo.rename(columns={'PobTot':'PobDist'},inplace=True)
ciudades_nuevo = ciudades_nuevo.fillna(0)
#7.Calculamos qué porcentaje representa la población de cada 'pedacito' de mancha urbana
#sobre el total de la población del distrito en el que se inserta.
#Podríamos asumir que si es más del 50%, ese 'pedacito' se toma en cuenta; de lo contrario, se elimina.
#Esto con la finalidad de evitar los 'cachitos' sobrantes de manchas urbanas.
ciudades_nuevo['PR_2']=ciudades_nuevo['PobParcial']/ciudades_nuevo['PobDist']*100
ciudades_nuevo.head()

,CODDPTO,NOMBDPTO,CIUDAD,layer,path,NOMBDIST,CODUBIGEO,IDPROV,NOMBPROV,ID,geometry,PobMU,PobParcial,PR,PobDist,PR_2
0,03,APURIMAC,ABANCAY,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,TAMBURCO,30109,0301,ABANCAY,1,"MULTIPOLYGON (((730284.004 8494704.893, 730288...",70313.0,8346.0,11.869782,10861.0,76.843753
1,03,APURIMAC,ABANCAY,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,ABANCAY,30101,0301,ABANCAY,2,"MULTIPOLYGON (((730726.099 8493090.736, 730727...",70313.0,61967.0,88.130218,69028.0,89.770818
2,03,APURIMAC,ANDAHUAYLAS,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,TALAVERA,30216,0302,ANDAHUAYLAS,3,"MULTIPOLYGON (((667519.696 8491559.078, 667530...",57236.0,11302.0,19.746314,18509.0,61.062186
3,03,APURIMAC,ANDAHUAYLAS,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,SAN JERONIMO,30213,0302,ANDAHUAYLAS,4,"MULTIPOLYGON (((676109.501 8490005.382, 676102...",57236.0,11579.0,20.230275,20738.0,55.834700
4,03,APURIMAC,ANDAHUAYLAS,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,ANDAHUAYLAS,30201,0302,ANDAHUAYLAS,5,"MULTIPOLYGON (((671729.911 8490283.357, 671731...",57236.0,34355.0,60.023412,42268.0,81.278982


In [33]:
ciudades_nuevo=ciudades_nuevo.drop(['layer','path'], axis=1)
ciudades_nuevo.to_file("data/5-ciudades_ok.geojson", driver='GeoJSON')

In [42]:
#8.¿Qué valores excluir? Puedo ir probando por percentiles
percentil=ciudades_nuevo['PR']
percentil = percentil.dropna(how='all')
np.percentile(percentil,q=[45]) #El P35 creo que es el más adecuado. El pedacito sobrante más grante lo encontré en
                                #la ciudad de La Merced, dpto. de Junín (PR=3.95)

array([8.28145614])

Con el geodataframe obtenido, después de analizar visualmente la información, creo que puede utilizarse un criterio de filtro que tome en cuenta los valores por encima del percentil 45 de la columna 'PR' (% poblacional que representa el 'pedacito' respecto del total de la mancha urbana, cuyo valor es 8.28145614) y por encima del 30% de la columna 'PR_2' (% poblacional que representa el 'pedacito' del total del distrito en el que se inserta).

In [13]:
#ciudades_nuevo=gpd.read_file('data/6-ciudades_ok.geojson') #en caso reinicie el kernel
import pandas as pd
import geopandas as gpd

ciudades_nuevo=gpd.read_file("data/5-ciudades_ok.geojson")
gdf_final=ciudades_nuevo[(ciudades_nuevo['PR']>=8.28145614) | (ciudades_nuevo['PR_2']>=30)]

Me interesa ahora ver si que existen ciudades que compartan distritos. Esto también podría ser problemático. En el mejor de los casos podrían unirse ciudades que compartan distritos para no excluirlas del análisis.

In [14]:
#1.Filtro para centrarme en las columnas CIUDAD y UBIGEO
filtro=gdf_final
filtro=filtro[['CIUDAD','CODUBIGEO']]
filtro.head()

,CIUDAD,CODUBIGEO
0,ABANCAY,30109
1,ABANCAY,30101
2,ANDAHUAYLAS,30216
3,ANDAHUAYLAS,30213
4,ANDAHUAYLAS,30201


In [15]:
#2. Encontramos las ciudades que comparten distritos.
filtro2=filtro[filtro['CODUBIGEO'].duplicated(keep=False)]
filtro2

,CIUDAD,CODUBIGEO
167,PISCO,110501
169,TUPAC AMARU,110501


![imagen](imagenes/ciudades_con_distritos_compartidos.jpeg)

Como puede verse, se trata de dos ciudades que acumulan 3 distritos. La delimitación sugiere que en estas ciudades se concentra la mayor parte de los tres distritos. Por tanto considero que, para no excluirlas del análisis, pueden considerarse como una sola ciudad.

In [16]:
#3. Reemplazamos TUPAC AMARU por PISCO, con la finalidad de que a partir de ahora sean consideradas
#como una sola ciudad
#3.1.Primero me fijo que no haya otra ciudad TUPAC AMARU, para eso veo su frecuencia. Debería ser 2 (porque
#está en dos distritos).
freq = filtro.groupby(['CIUDAD']).count() 
print(freq)

             CODUBIGEO
CIUDAD                
ABANCAY              2
ANDAHUAYLAS          3
AREQUIPA            17
AYACUCHO             5
AYAVIRI              1
...                ...
TUPAC AMARU          2
VIRU                 1
YAURI                1
YURIMAGUAS           1
ZARUMILLA            2

[92 rows x 1 columns]


In [17]:
gdf_final['CIUDAD']=gdf_final['CIUDAD'].replace(['TUPAC AMARU'], 'PISCO')

c:\programdata\miniconda3\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [19]:
freq = gdf_final.groupby(['CIUDAD']).count() 
print(freq) #Ya no aparece tupac amaru

             CODDPTO  NOMBDPTO  NOMBDIST  CODUBIGEO  IDPROV  NOMBPROV  ID  \
CIUDAD                                                                      
ABANCAY            2         2         2          2       2         2   2   
ANDAHUAYLAS        3         3         3          3       3         3   3   
AREQUIPA          17        17        17         17      17        17  17   
AYACUCHO           5         5         5          5       5         5   5   
AYAVIRI            1         1         1          1       1         1   1   
...              ...       ...       ...        ...     ...       ...  ..   
TUMBES             1         1         1          1       1         1   1   
VIRU               1         1         1          1       1         1   1   
YAURI              1         1         1          1       1         1   1   
YURIMAGUAS         1         1         1          1       1         1   1   
ZARUMILLA          2         2         2          2       2         2   2   

In [1]:
#4. Como paso final, vamos a eliminar duplicados. 
#Pasa que como TUPAC AMARU y PISCO compartían un distrito (PISCO, UBIGEO 110501), ese distrito va a salir duplicado
#y nos puede causar problemas más adelante.

#4.1.Primero comprobamos qué largo tiene este gdf
len(gdf_final)

246

In [10]:
#4.2.Ahora sí excluimos el duplicado
gdf_final2=gdf_final.drop_duplicates(['CODUBIGEO'])
len(gdf_final2)

245

In [11]:
#4.3.Exportamos
gdf_final2.to_file("data/6-ciudades_final.geojson", driver='GeoJSON')

gdf_final2.to_csv("data/6-ciudades_final.csv") #para trabajar en R